In [1]:
#Just some imports you will need
import mlflow
import jwt
import json
import warnings
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import logging
import os
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [2]:
print('MLFLOW_TRACKING_URI: ' + os.environ['MLFLOW_TRACKING_URI'])

MLFLOW_TRACKING_URI: http://127.0.0.1:8765


In [3]:
client = mlflow.tracking.MlflowClient()

In [4]:
# Here we are setting up the naming convention that we want to use for our MLFlow experiments. In this case it is a combination of our Project Name and Username.
myprefix=''

from datetime import datetime
now = datetime.now()
date_time_str = now.strftime("%m-%d-%Y")

experiment_name = 'demo'+'-' + os.environ['DOMINO_STARTING_USERNAME'] + '-' + os.environ['DOMINO_PROJECT_NAME']
model_name = 'demo'+'-' + os.environ['DOMINO_PROJECT_NAME']
if myprefix!='':
    experiment_name = myprefix + '-' + experiment_name
print(experiment_name)

demo-dev_uk1-flow_test


In [5]:
# Next we create the MLFlow experiment based on the name from the previous cell, unless it exists already. 
# This will be used to log all of our different model training runs so our team can review them at a later date.
experiment = client.get_experiment_by_name(name=experiment_name)
if(experiment is None):
    print('Creating experiment ')
    client.create_experiment(name=experiment_name)
    experiment = client.get_experiment_by_name(name=experiment_name)

print(experiment_name)
mlflow.set_experiment(experiment_name=experiment_name)

Creating experiment 
demo-dev_uk1-flow_test


<Experiment: artifact_location='mlflow-artifacts:/mlflow', creation_time=1715165912958, experiment_id='3', last_update_time=1715165912958, lifecycle_stage='active', name='demo-dev_uk1-flow_test', tags={'mlflow.domino.dataset_info': '663b5102631ba025af5714b4-663b5102631ba025af5714b3',
 'mlflow.domino.environment_id': '6615011314be3c4471a04f8f',
 'mlflow.domino.environment_revision_id': '6615011314be3c4471a04f94',
 'mlflow.domino.hardware_tier': 'small-k8s',
 'mlflow.domino.project_id': '663b5101631ba025af5714ae',
 'mlflow.domino.project_name': 'flow_test',
 'mlflow.domino.provenance_checkpoint_id': '663b55dd631ba025af5714cd',
 'mlflow.domino.run_id': '663b55c3631ba025af5714c8',
 'mlflow.domino.run_number': '1',
 'mlflow.domino.user': 'dev_uk1',
 'mlflow.domino.user_id': '66323b01631ba025af571306',
 'mlflow.source.git.branch': 'main',
 'mlflow.source.git.commit': 'a51bd320e12fd5b9cf7a3dbd8664a654ec85f80b',
 'mlflow.source.type': 'NOTEBOOK',
 'mlflow.user': 'dev_uk1'}>

In [6]:
# We define a function that will evaluate the effectiveness of our model 
# based on the actual data and the data predicted by the model.
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [7]:
# This cell is the main bulk of work - it will:
# Read in the data
# Prepare the data for training/testing
# Loop through different combinations of parameters
# For each parameter it will train the model, evaluate the model and log both the parameters and model.


# Read the wine-quality csv file from the URL
csv_url = (
     "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
)
try:
     data = pd.read_csv(csv_url, sep=";")
except Exception as e:
     logger.exception(
          "Unable to download training & test CSV, check your internet connection. Error: %s", e
      )

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]


my_log = "This is a test log"
with open("/tmp/test.txt", 'w') as f:
    f.write(my_log)
with open("/tmp/test.log", 'w') as f:
    f.write(my_log)

    
    
#run_tags={'mlflow.user':os.environ['DOMINO_STARTING_USERNAME']}
#Change user name
alpha = 0.7
l1_ratio = 0.6
while(alpha<1):
    with mlflow.start_run():
        print('Start Run')
        print('Alpha : ' + str(alpha))
        print('L1_Ratio : ' + str(l1_ratio))
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        print(mlflow.get_tracking_uri())
        print(tracking_url_type_store)
        # Model registry does not work with file store
        if tracking_url_type_store != "file":
            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            #mlflow.sklearn.log_model(lr, os.environ['DOMINO_STARTING_USERNAME']+"/model", registered_model_name="DEMO-ElasticnetWineModel-11")
            mlflow.sklearn.log_model(lr, "demo_model", registered_model_name=model_name)

        else:        
            mlflow.sklearn.log_model(lr,  model_name)
        print(mlflow.get_artifact_uri())
        alpha=alpha+0.1
        l1_ratio = l1_ratio + 0.05        
        

Start Run
Alpha : 0.7
L1_Ratio : 0.6
Elasticnet model (alpha=0.700000, l1_ratio=0.600000):
  RMSE: 0.7567096992784073
  MAE: 0.6133554798635241
  R2: 0.03331542034162194
http://127.0.0.1:8765
http


Successfully registered model 'demo-flow_test'.
2024/05/08 10:58:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: demo-flow_test, version 1
Created version '1' of model 'demo-flow_test'.


mlflow-artifacts:/mlflow/c9f187925ce74871858c20cc70e09b54/artifacts
Start Run
Alpha : 0.7999999999999999
L1_Ratio : 0.65
Elasticnet model (alpha=0.800000, l1_ratio=0.650000):
  RMSE: 0.7577581274612739
  MAE: 0.6159517201070597
  R2: 0.03063486407408511
http://127.0.0.1:8765
http


/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'demo-flow_test' already exists. Creating a new version of this model...
2024/05/08 10:59:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: demo-flow_test, version 2
Created version '2' of model 'demo-flow_test'.


mlflow-artifacts:/mlflow/2be70caf17cc4a9a8d7192bdf52c1ad9/artifacts
Start Run
Alpha : 0.8999999999999999
L1_Ratio : 0.7000000000000001
Elasticnet model (alpha=0.900000, l1_ratio=0.700000):
  RMSE: 0.7579213411905277
  MAE: 0.6171503570834783
  R2: 0.03021723546159094
http://127.0.0.1:8765
http


/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'demo-flow_test' already exists. Creating a new version of this model...
2024/05/08 10:59:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: demo-flow_test, version 3
Created version '3' of model 'demo-flow_test'.


mlflow-artifacts:/mlflow/889e241df0ed45afa763b5ff842de574/artifacts
Start Run
Alpha : 0.9999999999999999
L1_Ratio : 0.7500000000000001
Elasticnet model (alpha=1.000000, l1_ratio=0.750000):
  RMSE: 0.7578758069260131
  MAE: 0.6181236903445718
  R2: 0.03033375683670947
http://127.0.0.1:8765
http


/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'demo-flow_test' already exists. Creating a new version of this model...
2024/05/08 10:59:07 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: demo-flow_test, version 4
Created version '4' of model 'demo-flow_test'.


mlflow-artifacts:/mlflow/714761cb8084498f84570c3d50d18d44/artifacts
